In [1]:
!pip install hvplot
!pip install holoviews

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import hvplot.pandas
import datetime as dt
import holoviews as hv
from prophet import Prophet

%matplotlib inline

Output hidden; open in https://colab.research.google.com to view.

In [3]:
from google.colab import files
uploaded = files.upload()

Saving scarf-google-trends-data.csv to scarf-google-trends-data.csv


In [4]:
# Read the data from the scarf-google-trends-data.csv file into a Pandas DataFrame
df_alpaca = pd.read_csv("scarf-google-trends-data.csv")

# Review the DataFrame
df_alpaca.head()

,week,canada,uruguay
0,11/8/15,74,17
1,11/15/15,81,17
2,11/22/15,94,17
3,11/29/15,92,0
4,12/6/15,87,18


In [5]:
# Set `bokeh` to render hvPlot charts
hv.extension("bokeh")

# Plot the general tredns using hvPlot
df_alpaca.hvplot() 

Output hidden; open in https://colab.research.google.com to view.

In [6]:
# Create a DataFrame for Canada to include the week and canada columns
df_canada = df_alpaca[["week", "canada"]]

# Rename the columns to the Prophet model syntax 
df_canada = df_canada.rename(columns={"week":"ds", "canada":"y"})

# Review the Canada DataFrame
df_canada.head()

,ds,y
0,11/8/15,74
1,11/15/15,81
2,11/22/15,94
3,11/29/15,92
4,12/6/15,87


In [7]:
# Create a DataFrame for Uruguay to include the week and uruguay columns
df_uruguay = df_alpaca[["week", "uruguay"]]

# Rename the columns to the Prophet model syntax 
df_uruguay = df_uruguay.rename(columns={"week":"ds", "uruguay":"y"})

# Review the Uruguay DataFrame
df_uruguay.head()

,ds,y
0,11/8/15,17
1,11/15/15,17
2,11/22/15,17
3,11/29/15,0
4,12/6/15,18


In [8]:
# Create a Prophet model for Canada
model_canada = Prophet()

In [9]:
model_uruguay = Prophet()

In [10]:
model_canada.fit(df_canada)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0itwmyqu/p4wb88g8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0itwmyqu/l2frojlo.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=75332', 'data', 'file=/tmp/tmp0itwmyqu/p4wb88g8.json', 'init=/tmp/tmp0itwmyqu/l2frojlo.json', 'output', 'file=/tmp/tmp0itwmyqu/prophet_model3bfo09x3/prophet_model-20221207010802.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:08:02 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:08:02 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [11]:
model_uruguay.fit(df_uruguay)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0itwmyqu/6_r9mdop.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0itwmyqu/zf0pk1wd.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.8/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=24409', 'data', 'file=/tmp/tmp0itwmyqu/6_r9mdop.json', 'init=/tmp/tmp0itwmyqu/zf0pk1wd.json', 'output', 'file=/tmp/tmp0itwmyqu/prophet_modelxfqruvie/prophet_model-20221207010814.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
01:08:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
01:08:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [12]:
# Forecast one year of weekly future trends data for Canada 
future_canada = model_canada.make_future_dataframe(periods=52, freq="W")

# Display the last five rows of the future_canada DataFrame
future_canada.tail()

,ds
307,2021-09-26
308,2021-10-03
309,2021-10-10
310,2021-10-17
311,2021-10-24


In [13]:
# Forecast one year of weekly future trends data for Uruguay 
future_uruguay = model_uruguay.make_future_dataframe(periods=52, freq="W")

# Display the last five rows of the future_uruguay DataFrame
future_uruguay.tail()

,ds
307,2021-09-26
308,2021-10-03
309,2021-10-10
310,2021-10-17
311,2021-10-24


In [14]:
# Make predictions for Canada using the future_canada DataFrame
forecast_canada = model_canada.predict(future_canada)

# Display the first five rows of the forecast_canada DataFrame
forecast_canada.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2015-11-08,46.024000,68.045656,81.015952,46.024000,46.024000,28.538662,28.538662,28.538662,28.538662,28.538662,28.538662,0.0,0.0,0.0,74.562662
1,2015-11-15,45.927797,72.088601,85.100542,45.927797,45.927797,32.938599,32.938599,32.938599,32.938599,32.938599,32.938599,0.0,0.0,0.0,78.866395
2,2015-11-22,45.831593,75.625797,88.050066,45.831593,45.831593,36.030354,36.030354,36.030354,36.030354,36.030354,36.030354,0.0,0.0,0.0,81.861947
3,2015-11-29,45.735390,78.739782,90.822505,45.735390,45.735390,38.872431,38.872431,38.872431,38.872431,38.872431,38.872431,0.0,0.0,0.0,84.607820
4,2015-12-06,45.639186,81.108960,93.965066,45.639186,45.639186,41.706148,41.706148,41.706148,41.706148,41.706148,41.706148,0.0,0.0,0.0,87.345334


In [15]:
# Make predictions for Uruguay using the future_uruguay DataFrame
forecast_uruguay = model_uruguay.predict(future_uruguay)

# Display the first five rows of the forecast_uruguay DataFrame
forecast_uruguay.head()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2015-11-08,33.346600,8.931607,45.722607,33.346600,33.346600,-7.082842,-7.082842,-7.082842,-7.082842,-7.082842,-7.082842,0.0,0.0,0.0,26.263758
1,2015-11-15,33.301427,6.045282,40.734191,33.301427,33.301427,-10.347929,-10.347929,-10.347929,-10.347929,-10.347929,-10.347929,0.0,0.0,0.0,22.953498
2,2015-11-22,33.256253,-0.290016,35.178603,33.256253,33.256253,-15.895338,-15.895338,-15.895338,-15.895338,-15.895338,-15.895338,0.0,0.0,0.0,17.360915
3,2015-11-29,33.211080,-4.029277,30.722986,33.211080,33.211080,-19.477105,-19.477105,-19.477105,-19.477105,-19.477105,-19.477105,0.0,0.0,0.0,13.733975
4,2015-12-06,33.165907,-2.957204,31.735826,33.165907,33.165907,-18.472699,-18.472699,-18.472699,-18.472699,-18.472699,-18.472699,0.0,0.0,0.0,14.693208
